In [205]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from glob import glob
import torchvision
import tqdm

In [206]:
class customdataset(Dataset): 
    def __init__(self, data_dir, transform= None): 
        #이닛 해주고
        self.data_dir= data_dir 
        self.transform= transform
        #이미지들 경로명 담을 리스트
        self.image_paths= []
        self.labels= []

        # NORMAL 폴더의 이미지 경로와 라벨 (0)
        normal_dir = os.path.join(data_dir, 'NORMAL')
        for img_name in os.listdir(normal_dir):
            self.image_paths.append(os.path.join(normal_dir, img_name))
            self.labels.append(0)

        # PNEUMONIA 폴더의 이미지 경로와 라벨 (1)
        pneumonia_dir = os.path.join(data_dir, 'PNEUMONIA')
        for img_name in os.listdir(pneumonia_dir):
            self.image_paths.append(os.path.join(pneumonia_dir, img_name))
            self.labels.append(1)

            
    def __len__(self): 
        #len이건 필수래 나중에 DataLoader 하려면 
        #몇번 반복해서 받을건지를 결정하는 요소? 
        #이미지파일 갯수만큼 받아야하니까
        return len(self.image_paths)

    def __getitem__(self, idx): 
        #이것도 필수래 DataLoader 에서 이터레이터한 객체를 받아올 수 있다는데? 
        #그래서 for문같은 반복문 추가 안해줘도 DataLoader에서 알아서 반복 해준대 ㅇㅇ
        #이미지 경로를 받아서 실제 이미지를 열어주는 구간
        
        image = Image.open(self.image_paths[idx]).convert('L')
        label= self.labels[idx]

        if self.transform: 
            image= self.transform(image)

        return image, label


In [207]:
train_transform= transforms.Compose([ 
    #데이터 전처리 하기위해서 필수인 과정 
    #데이터 증강도 할 수 있음 +a인 느낌 
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),  # 수평 뒤집기
    transforms.RandomVerticalFlip(),    # 수직 뒤집기 추가
    transforms.RandomRotation(30),      # 더 강한 회전 추가
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),  # 더 강한 색상 변형
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2)),  # 더 강한 이동 변환
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

val_transform= transforms.Compose([
    #검증 데이터셋을 만들 땐 전처리만
    #데이터 증강은 필요없음. 
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=1), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.5], std=[0.5]) 
])


In [208]:
full_dataset= customdataset(data_dir= 'chest_xray/train', transform= train_transform) 
train_size= int(0.8*len(full_dataset)) 
val_size= len(full_dataset) - train_size
train_dataset, val_dataset= random_split(full_dataset, [train_size, val_size]) 


In [209]:
len(full_dataset)

5216

In [210]:
print(full_dataset.transform, train_dataset.dataset.transform)

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    RandomHorizontalFlip(p=0.5)
    RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
    ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=None, hue=None)
    RandomAffine(degrees=[0.0, 0.0], translate=(0.1, 0.1))
    ToTensor()
    Normalize(mean=[0.5], std=[0.5])
) Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    RandomHorizontalFlip(p=0.5)
    RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
    ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=None, hue=None)
    RandomAffine(degrees=[0.0, 0.0], translate=(0.1, 0.1))
    ToTensor()
    Normalize(mean=[0.5], std=[0.5])
)


In [211]:

# 검증 데이터는 데이터 증강을 적용하지 않기 때문에 변환을 따로 지정
val_dataset.dataset = customdataset(data_dir='chest_xray/train', transform=val_transform)


In [212]:
print(full_dataset.transform, train_dataset.dataset.transform, val_dataset.dataset.transform)

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    RandomHorizontalFlip(p=0.5)
    RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
    ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=None, hue=None)
    RandomAffine(degrees=[0.0, 0.0], translate=(0.1, 0.1))
    ToTensor()
    Normalize(mean=[0.5], std=[0.5])
) Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    RandomHorizontalFlip(p=0.5)
    RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
    ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=None, hue=None)
    RandomAffine(degrees=[0.0, 0.0], translate=(0.1, 0.1))
    ToTensor()
    Normalize(mean=[0.5], std=[0.5])
) Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    Grayscale(num_output_channels=1)
    ToTensor()
    Normalize(mean=[0.5], std=[0.5

In [213]:
# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)


In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.downsample = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = self.downsample(x)
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual
        out = self.relu(out)
        return out

class EnhancedResNet(nn.Module):
    def __init__(self):
        super(EnhancedResNet, self).__init__()
        self.layer1 = ResidualBlock(1, 32)
        self.layer2 = ResidualBlock(32, 64, stride=2)
        self.layer3 = ResidualBlock(64, 128, stride=2)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(128, 2)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x


In [214]:
class EnhancedCNN(nn.Module): 
    def __init__(self): 
        #초기화, 변수 설정해주는 구간 
        super(EnhancedCNN, self).__init__()
        self.conv1= nn.Conv2d(1, 32, kernel_size= 3, stride= 1, padding= 1) 
        self.bn1= nn.BatchNorm2d(32) 
        self.conv2= nn.Conv2d(32, 64, kernel_size= 3, stride= 1, padding= 1) 
        self.bn2= nn.BatchNorm2d(64) 
        self.conv3= nn.Conv2d(64, 128, kernel_size= 3, stride= 1, padding= 1) 
        self.bn3= nn.BatchNorm2d(128)
        self.pool= nn.MaxPool2d(kernel_size= 2, stride= 2, padding= 0) 
        self.dropout= nn.Dropout(0.5) 
        self.fc1= nn.Linear(128*28*28, 256)
        self.fc2= nn.Linear(256, 2) 

    def forward(self, x): 
        #여기가 실제로 층 쌓는 구간
        x= self.pool(F.relu(self.bn1(self.conv1(x))))
        x= self.pool(F.relu(self.bn2(self.conv2(x)))) 
        x= self.pool(F.relu(self.bn3(self.conv3(x)))) 
        x= x.view(-1, 128*28*28)
        x= F.relu(self.fc1(x)) 
        x= self.dropout(x) 
        x= self.fc2(x) 
        return x

In [215]:
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model= EnhancedResNet().to(device) 
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
criterion= nn.CrossEntropyLoss()
num_epochs= 50
# 조기 종료 조건 설정
patience = 5
best_val_loss = float('inf')
early_stop_counter = 0

In [216]:
for epoch in range(num_epochs): 
    #초기화 해주는거 
    #학습모드로 변경
    model.train() 
    running_loss= 0.0

    #얘네는 recall 구하기 위해서 추가한거
    all_train_labels= []
    all_train_predictions= [] 
    #정확도 구할 때 씀 
    correct= 0 
    total= 0 


    for images, labels in tqdm.tqdm(train_loader): 
        #gpu 디바이스로 적용해서 진행하려고 to(device) 
        images, labels= images.to(device), labels.to(device) 

        optimizer.zero_grad() 

        outputs= model(images) 
        loss= criterion(outputs, labels) 

        #recall 구하려고 하는거 예측값 저장
        _, predicted= torch.max(outputs.data, 1)
        all_train_labels.extend(labels.cpu().numpy())
        all_train_predictions.extend(predicted.cpu().numpy()) 

        #정확도 구하자
        total+= labels.size(0) 
        correct += (predicted == labels).sum().item()

        loss.backward() 
        optimizer.step() 

        running_loss+= loss.item()
    #손실 출력 
    accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}, Train Accuracy: {accuracy:.2f}%')

    #recall 값 출력 
    train_report= classification_report(all_train_labels, all_train_predictions, target_names= ['NORMAL', 'PNEUMONIA'], output_dict= True) 
    train_recall= train_report['weighted avg']['recall']
    print(f'train_recall: {train_recall:.4f}')



    #val 상단에 해놓은거랑 똑같이 하면 댐
    model.eval() 
    val_loss = 0.0
    all_val_labels= []
    all_val_predictions= [] 
    

    correct= 0 
    total= 0 

    #검증용이라서 역전파 없이 함
    with torch.no_grad(): 
        for images, labels in val_loader: 
            images, labels= images.to(device), labels.to(device)
            
            outputs= model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()


            #recall 구하려고 하는거 예측값 저장
            _, predicted= torch.max(outputs.data, 1) 
            all_val_labels.extend(labels.cpu().numpy())
            all_val_predictions.extend(predicted.cpu().numpy())
    
            #정확도 구하자
            total+= labels.size(0) 
            correct += (predicted == labels).sum().item()

    # 검증 데이터 예측 지표 출력
    val_report = classification_report(all_val_labels, all_val_predictions, target_names=['NORMAL', 'PNEUMONIA'], output_dict=True)
    val_weighted_avg_recall = val_report['weighted avg']['recall']
    print(f"val_recall: {val_weighted_avg_recall:.4f}")

    # 정확도 출력
    accuracy = 100 * correct / total
    print(f'Loss: {val_loss / len(val_loader):.4f}, Validation Accuracy: {accuracy:.2f}%')

    # 학습률 스케줄러 스텝
    scheduler.step()

    # 조기 종료(Early Stopping) 체크
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

            

Epoch [1/10], Loss: 0.4032, Train Accuracy: 81.30%
train_recall: 0.8130
val_recall: 0.9023
Loss: 1.6007, Validation Accuracy: 90.23%
Epoch [2/10], Loss: 0.2966, Train Accuracy: 87.37%
train_recall: 0.8737
val_recall: 0.9310
Loss: 1.1775, Validation Accuracy: 93.10%
Epoch [3/10], Loss: 0.2422, Train Accuracy: 89.98%
train_recall: 0.8998
val_recall: 0.9301
Loss: 0.9613, Validation Accuracy: 93.01%
Epoch [4/10], Loss: 0.2178, Train Accuracy: 91.51%
train_recall: 0.9151
val_recall: 0.9540
Loss: 0.8645, Validation Accuracy: 95.40%
Epoch [5/10], Loss: 0.2248, Train Accuracy: 91.08%
train_recall: 0.9108
val_recall: 0.9157
Loss: 0.8924, Validation Accuracy: 91.57%
Epoch [6/10], Loss: 0.1904, Train Accuracy: 92.69%
train_recall: 0.9269
val_recall: 0.8812
Loss: 0.7558, Validation Accuracy: 88.12%
Epoch [7/10], Loss: 0.2033, Train Accuracy: 91.73%
train_recall: 0.9173
val_recall: 0.9473
Loss: 0.8072, Validation Accuracy: 94.73%
Epoch [8/10], Loss: 0.1934, Train Accuracy: 93.05%
train_recall: 0.93

In [217]:
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

test_dataset = customdataset(data_dir='chest_xray/test', transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# 모델 평가 모드로 설정
model.eval()
all_labels = []
all_predictions = []
total= 0
correct= 0

# 테스트 데이터셋에 대한 예측 및 실제 라벨 저장
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        
        
        outputs = model(images)
        
        
        # recall 구하려고 실제 라벨과 예측 라벨을 리스트에 추가
        _, predicted = torch.max(outputs.data, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())


        #정확도 구하자
        total+= labels.size(0) 
        correct += (predicted == labels).sum().item()

# Classification Report 출력 및 Weighted Average Recall 추출
report = classification_report(all_labels, all_predictions, target_names=['NORMAL', 'PNEUMONIA'], output_dict=True)

# 출력 보고서에서 weighted avg recall 값 추출
weighted_avg_recall = report['weighted avg']['recall']
print("Classification Report:")
print(classification_report(all_labels, all_predictions, target_names=['NORMAL', 'PNEUMONIA']))
print(f"Weighted Average Recall: {weighted_avg_recall:.4f}")

# 정확도 출력
accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.4f}%')



Classification Report:
              precision    recall  f1-score   support

      NORMAL       0.80      0.86      0.83       242
   PNEUMONIA       0.91      0.87      0.89       398

    accuracy                           0.87       640
   macro avg       0.86      0.86      0.86       640
weighted avg       0.87      0.87      0.87       640

Weighted Average Recall: 0.8656
Accuracy: 86.5625%
